# Network Security — Exercise 5: Domain Name System Security & Network Firewalls

**Noah Link, Jan Pfeifer, Julian Weske**  
**Date:** 20.06.2024

## Domain Name System Security (Time spent: xx h)

### Investigating Legal Websites

Initializing the project and dnsdbq worked just fine:
```
./dnsdbq -u circl -n www.heise.de
;; record times: 2023-10-08 18:53:22 .. 2024-06-12 16:29:03 (~247d 21h 35m)
;; count: 369
193.99.144.85  A  www.heise.de

... skipped ...

;; record times: 2024-01-21 15:44:12 .. 2024-06-11 23:03:38 (~142d 7h 19m)
;; count: 7
www.heise.de  PTR  85.144.99.193.in-addr.arpa
```

### Passive DNS

- **What is passive DNS?**  
  Passive DNS is a method of collecting, storing, and analyzing DNS query and response data from recursive DNS servers. This allows for historical tracking of domain-to-IP mappings without actively querying DNS servers.

- **How can investigations in cybercrime benefit from passive DNS analysis?**
  - Identifying historical domain-IP mappings to uncover the infrastructure used.
  - Tracking changes in domain associations, which can reveal patterns of malicious activity.
  
- **Name two factors that the quality of passive DNS analysis, i.e., the number of returned results, depends on.**
  - The volume and diversity of DNS data sources contributing to the passive DNS database.
  - The time span over which DNS data has been collected and stored.

## IT Security News
Please use passive DNS to analyze the legal website www.heise.de according to the following questions:

In [2]:
import pypdns
auth_details = ('students.informatik.uni-hamburg.de', 'DWW/ymamruvwjRfwo8g8SFaCw1H8zYj5GlxBS8JVWgM=')
client = pypdns.PyPDNS(basic_auth=auth_details)
host = "www.heise.de"


1. How many IPv4 addresses exist for this site? List all addresses (i.e., A records)
you found:

In [3]:
# 1.1.2.1
result = client.rfc_query(host)# , filter_rrtype='A') #todo 
ip_addresses = [record.rdata for record in result]

print("IP addresses for ", host, ": ")
for address in ip_addresses:
    print(address)

IP addresses for  www.heise.de : 
5.8.0.0.2.0.0.0.e.2.7.7.7.7.7.7.1.0.0.1.e.f.3.0.0.e.2.0.2.0.a.2.ip6.arpa
karriere.heise-gruppe.de
monocle.heise.de
www.heisegroup.de
compaliate.heise.de
85.144.99.193.in-addr.arpa
reichweite.heise.de
www.heise.de
www.heise.de
www.heise.de
www.heise.de
www.heise.de
www.telepolis.de
www.heise.de
www.heise.de


2. Is this site using IPv6 as well? If yes, list all addresses (i.e., AAAA records) you
found.

In [5]:
# 1.1.2.2
result = client.rfc_query(host, filter_rrtype='AAAA')
ip_addresses = [record.rdata for record in result]

print("IP addresses for ", host, ": ")
for address in ip_addresses:
    print(address)

IP addresses for  www.heise.de : 
www.heise.de


3. Search for other Fully-Qualified Domain Names (FQDNs) that are also hosted on
the first IPv4 address of www.heise.de.

In [6]:
# 1.1.2.3
results = client.rfc_query(host)

fqdn_set = set()
for result in results:
    if result.rrname:
        fqdn_set.add(result.rrname)

print(f'FQDNs hosted on the same IP address ({host}):')
for fqdn in fqdn_set:
    print(fqdn)

FQDNs hosted on the same IP address (www.heise.de):
tiktok-developers-site-verification=weIoKWwdQgrTvpnCHhaQYbSHVDYzRrPG
www.heise.de
193.99.144.85
tiktok-developers-site-verification=FHHUbiCIOtixTBCGm2NCq7EWqXVFDha9
tiktok-developers-site-verification=4hG7PLWNR1XvEcOT4YiLagaIG0dISTp5
2a02:2e0:3fe:1001:7777:772e:2:85
tiktok-developers-site-verification=ItHbW0BcRfQvyRg64ke5Hjyr8Kq0tNLw
tiktok-developers-site-verification=XRVQfkPuANzTVYCOWuMGUoo3cuGojxoC


4. List all FQDNs that share a common IPv6 address with www.heise.de.

In [7]:
# 1.1.2.4
import socket
result = client.rfc_query(host, filter_rrtype='AAAA')
ipv6_addresses = {record.rdata for record in result}

for ipv6_address in ipv6_addresses:
    query_results = client.rfc_query(ipv6_address, filter_rrtype='AAAA')
    
    if result:
        fqdns = {record.rrname for record in query_results}
        fqdns.discard(host)
        print(f"FQDNs sharing IPv6 address {ipv6_address}:")
        for fqdn in fqdns:
            print(fqdn)
    else:
        print(f"No FQDNs found sharing IPv6 address {ipv6_address}")

    try:
        fqdn = socket.gethostbyaddr(ipv6_address)
        print(f"The FQDN of {ipv6_address} is: {fqdn}")
    except socket.herror as e:
        print(f"Unable to find the FQDN for {ipv6_address}")


FQDNs sharing IPv6 address www.heise.de:
2a02:2e0:3fe:1001:7777:772e:2:85
The FQDN of www.heise.de is: ('www.heise.de', ['85.144.99.193.in-addr.arpa'], ['193.99.144.85'])


### University Website
Please use passive DNS to analyze the legal website www.uni-hamburg.de.
Hint: For this task, you might not be able to find a directly referenced A record, but CNAME records instead that you have to follow. In this case, continue with the most recent alias name

1. How many IPv4 addresses exist for this site? List all addresses (i.e., A records) you found.

In [6]:
# 1.1.3.1
host = "www.uni-hamburg.de"
result = client.rfc_query(host)# , filter_rrtype='A')
ip_addresses = [record.rdata for record in result]

print("IP addresses for ", host, ": ")
for address in ip_addresses:
    print(address)

IP addresses for  www.uni-hamburg.de : 
www.uni-hamburg.de


2. Is this site using IPv6 as well? If yes, list all addresses (i.e., AAAA records) you found.

In [7]:
# 1.1.3.2
result = client.rfc_query(host, filter_rrtype='AAAA')
ip_addresses = [record.rdata for record in result]

print("IP addresses for ", host, ": ")
for address in ip_addresses:
    print(address)

IP addresses for  www.uni-hamburg.de : 



3. Search for other Fully-Qualified Domain Names (FQDNs) that are also hosted on the first IPv4 address of www.uni-hamburg.de.

In [8]:
# 1.1.3.3
import pypdns
auth_details = ('students.informatik.uni-hamburg.de', 'DWW/ymamruvwjRfwo8g8SFaCw1H8zYj5GlxBS8JVWgM=')
client = pypdns.PyPDNS(basic_auth=auth_details)
host = "www.heise.de"
results = client.rfc_query(host)

fqdn_set = set()
for result in results:
    if result.rrname:
        fqdn_set.add(result.rrname)

print(f'FQDNs hosted on the same IP address ({host}):')
for fqdn in fqdn_set:
    print(fqdn)

FQDNs hosted on the same IP address (www.heise.de):
www.heise.de
tiktok-developers-site-verification=4hG7PLWNR1XvEcOT4YiLagaIG0dISTp5
tiktok-developers-site-verification=weIoKWwdQgrTvpnCHhaQYbSHVDYzRrPG
tiktok-developers-site-verification=ItHbW0BcRfQvyRg64ke5Hjyr8Kq0tNLw
tiktok-developers-site-verification=XRVQfkPuANzTVYCOWuMGUoo3cuGojxoC
2a02:2e0:3fe:1001:7777:772e:2:85
tiktok-developers-site-verification=FHHUbiCIOtixTBCGm2NCq7EWqXVFDha9
193.99.144.85



4. List all FQDNs that share a common IPv6 address with www.uni-hamburg.de.

In [9]:
# 1.1.3.4
# Todo sollte eigentlich funktionieren aber ipv6_addresses_uni ist leer
host = "www.uni-hamburg.de"

import socket
result = client.rfc_query(host, filter_rrtype='AAAA')
ipv6_addresses = {record.rdata for record in result}

for ipv6_address in ipv6_addresses:
    query_results = client.rfc_query(ipv6_address, filter_rrtype='AAAA')
    
    if result:
        fqdns = {record.rrname for record in query_results}
        fqdns.discard(host)
        print(f"FQDNs sharing IPv6 address {ipv6_address}:")
        for fqdn in fqdns:
            print(fqdn)
    else:
        print(f"No FQDNs found sharing IPv6 address {ipv6_address}")

    try:
        fqdn = socket.gethostbyaddr(ipv6_address)
        print(f"The FQDN of {ipv6_address} is: {fqdn}")
    except socket.herror as e:
        print(f"Unable to find the FQDN for {ipv6_address}")

5. Find more domain names that do not directly resolve to the IP address of www.uni-hamburg.de but also indirectly via the respective CNAME.

In [14]:
# 1.1.3.5
# todo: check if correct. der kernel crashed hier ab und zu :c
import dns.resolver

try:
    cname_records = dns.resolver.resolve(host, 'CNAME')
    for record in cname_records:
        alias = record.target.to_text().rstrip('.')
        print(f"{host} CNAME -> {alias}")
        while True:
            try:
                cname_records = dns.resolver.resolve(alias, 'CNAME')
                for record in cname_records:
                    alias = record.target.to_text().rstrip('.')
                    print(f"{alias} CNAME -> {alias}")
                aaaa_records = dns.resolver.resolve(alias, 'AAAA')
                if aaaa_records:
                    ipv6_addresses = {record.to_text() for record in aaaa_records}
                    print(f"{alias} AAAA -> {ipv6_addresses}")
                else:
                    print(f"No AAAA records found for {alias}")

                break 
            except dns.resolver.NoAnswer:
                print(f"No CNAME records found for {alias}")
                break  
except dns.resolver.NoAnswer:
    print(f"No CNAME records found for {host}")
except dns.resolver.NXDOMAIN:
    print(f"Domain {host} does not exist")

www.uni-hamburg.de CNAME -> www-fiona.rrz.uni-hamburg.de
No CNAME records found for www-fiona.rrz.uni-hamburg.de


### Investigating Illegal Websites
Please use passive DNS to analyze the illegal streaming website kinox.to.
Hint: This website may not exist anymore, so you can also investigate other illegal websites. You presume that more streaming websites are hosted within the same address block.

We chose to do our analysis on the ‘burning-series.to’ domain, a more current example compared to ‘kinox.to’. It presented us with quite a lot of related domains.

First we take a look at domains using the same ip addresses, that burning-series.to used in the past:

In [2]:
from pypdns import PyPDNS

auth_details = ('students.informatik.uni-hamburg.de', 'DWW/ymamruvwjRfwo8g8SFaCw1H8zYj5GlxBS8JVWgM=')
client = PyPDNS(basic_auth=auth_details)

domain = 'burning-series.to'
results = client.query(domain)

results





[{'rrtype': 'A',
  'rrname': '104.21.71.4',
  'rdata': 'burning-series.to',
  'count': 2,
  'time_first': datetime.datetime(2023, 11, 7, 22, 52),
  'time_last': datetime.datetime(2023, 11, 21, 13, 35, 10),
  'origin': 'https://www.circl.lu/pdns/'},
 {'rrtype': 'A',
  'rrname': '172.67.141.64',
  'rdata': 'burning-series.to',
  'count': 2,
  'time_first': datetime.datetime(2023, 11, 7, 22, 52),
  'time_last': datetime.datetime(2023, 11, 21, 13, 35, 10),
  'origin': 'https://www.circl.lu/pdns/'},
 {'rrtype': 'NS',
  'rrname': 'liz.ns.cloudflare.com',
  'rdata': 'burning-series.to',
  'count': 1,
  'time_first': datetime.datetime(2023, 11, 24, 15, 20, 50),
  'time_last': datetime.datetime(2023, 11, 24, 15, 20, 50),
  'origin': 'https://www.circl.lu/pdns/'},
 {'rrtype': 'NS',
  'rrname': 'jack.ns.cloudflare.com',
  'rdata': 'burning-series.to',
  'count': 1,
  'time_first': datetime.datetime(2023, 11, 24, 15, 20, 50),
  'time_last': datetime.datetime(2023, 11, 24, 15, 20, 50),
  'origin': 

In [3]:
ip_addresses = [result['rrname'] for result in results if result['rrtype'] == 'A']

# Display the IP addresses
ip_addresses



['104.21.71.4', '172.67.141.64', '199.59.243.225']

Then we query for FQDNs associated with those IPs:

In [29]:
fqdn_results = []
for ip in ip_addresses:
    print(f"Querying FQDNs associated with IP: {ip}")
    try:
        ip_results = client.query(ip)
        fqdn_results.extend(ip_results)
    except PDNSError as e:
        logging.error(f"Error querying IP {ip}: {e}")
        continue

    # Display the FQDN results
    print(fqdn_results)



Querying FQDNs associated with IP: 104.21.71.4
[{'rrtype': 'A', 'rrname': '104.21.71.4', 'rdata': 'dimplex.electricfireplaceq.com', 'count': 1, 'time_first': datetime.datetime(2023, 10, 20, 11, 20, 16), 'time_last': datetime.datetime(2023, 10, 20, 11, 20, 16), 'origin': 'https://www.circl.lu/pdns/'}, {'rrtype': 'A', 'rrname': '104.21.71.4', 'rdata': 'ashley.electricfireplaceq.com', 'count': 1, 'time_first': datetime.datetime(2023, 10, 20, 11, 20, 16), 'time_last': datetime.datetime(2023, 10, 20, 11, 20, 16), 'origin': 'https://www.circl.lu/pdns/'}, {'rrtype': 'A', 'rrname': '104.21.71.4', 'rdata': 'es.forkord-turnaround.com', 'count': 1, 'time_first': datetime.datetime(2023, 10, 21, 10, 1, 43), 'time_last': datetime.datetime(2023, 10, 21, 10, 1, 43), 'origin': 'https://www.circl.lu/pdns/'}, {'rrtype': 'A', 'rrname': '104.21.71.4', 'rdata': 'nextlvltee.com', 'count': 1, 'time_first': datetime.datetime(2023, 10, 29, 12, 22, 1), 'time_last': datetime.datetime(2023, 10, 29, 12, 22, 1), 'or

In [30]:
# filter out meaningless results by applying search strings
search_strings = ['stream', 'video', 'movie', 'watch', 'tv']

filtered_fqdns = []
for fqdn in fqdn_results:
        for string in search_strings:
            if string in fqdn['rdata']:
                filtered_fqdns.append({'rdata': fqdn['rdata'], 'rrname': fqdn['rrname']})
                break

# Display the filtered FQDNs
print("Filtered FQDNs:")
for fqdn in filtered_fqdns:
    print(f"URL: {fqdn['rdata']}, IP: {fqdn['rrname']}")

Filtered FQDNs:
URL: 8m7tv3.69av295.xyz, IP: 104.21.71.4
URL: www.bienstream.net, IP: 104.21.71.4
URL: zntv177.top, IP: 104.21.71.4
URL: goatvip168.com, IP: 104.21.71.4
URL: www.goatvip168.com, IP: 104.21.71.4
URL: www.justwatchit.co, IP: 104.21.71.4
URL: justwatchit.co, IP: 104.21.71.4
URL: lnhntv8324.top, IP: 104.21.71.4
URL: www.tvpn.trading1093.workers.dev, IP: 104.21.71.4
URL: 8m7tv3.69av295.xyz, IP: 172.67.141.64
URL: www.bienstream.net, IP: 172.67.141.64
URL: zntv177.top, IP: 172.67.141.64
URL: goatvip168.com, IP: 172.67.141.64
URL: www.goatvip168.com, IP: 172.67.141.64
URL: www.justwatchit.co, IP: 172.67.141.64
URL: justwatchit.co, IP: 172.67.141.64
URL: lnhntv8324.top, IP: 172.67.141.64
URL: www.tvpn.trading1093.workers.dev, IP: 172.67.141.64
URL: 0cwjx6r1fl.aniewatch.to, IP: 199.59.243.225
URL: 0123movies4kuhd.hyperphp.com, IP: 199.59.243.225
URL: 0123movies.li, IP: 199.59.243.225
URL: 0-j.us.tv, IP: 199.59.243.225
URL: 086559ypsd.magyarorszag.hu.tv, IP: 199.59.243.225
URL: 0

That yields 58 domains probably affiliated with illegal streaming activity,

Finally we can scan through the whole address ranges of the IPs we obtained, to get even more results:

In [4]:
#same steps as before
import ipaddress
from pypdns import PyPDNS

auth_details = ('students.informatik.uni-hamburg.de', 'DWW/ymamruvwjRfwo8g8SFaCw1H8zYj5GlxBS8JVWgM=')
domain = 'burning-series.to'

client = PyPDNS(basic_auth=auth_details)
results = client.query(domain) 

# obtain ips used by the domain in the past
ip_addresses = {result['rrname'] for result in results if result['rrtype'] == 'A'}

def get_ip_range(ip):
    return [str(ip) for ip in ipaddress.IPv4Network(f"{ip}/24", strict=False)]

# generate set of all ips in the /24 range for each ip
all_ips = set()
for ip in ip_addresses:
    all_ips.update(get_ip_range(ip))

results = []
# query the database for each ip
for ip in all_ips:
    results.extend(client.query(ip))

search_strings = {'stream', 'video', 'movie', 'watch', 'tv'} 

# filter results by search strings
filtered_fqdns = [
    {'rdata': fqdn['rdata'], 'rrname': fqdn['rrname']}
    for fqdn in results
    if any(string in fqdn['rdata'] for string in search_strings)
]

# Display the filtered FQDNs
print("Filtered FQDNs:")
for fqdn in filtered_fqdns:
    print(f"URL: {fqdn['rdata']}, IP: {fqdn['rrname']}")

Filtered FQDNs:
URL: viralvideotube.us, IP: 104.21.71.198
URL: www.androiptvapp.com, IP: 172.67.141.154
URL: androiptvapp.com, IP: 172.67.141.154
URL: www.wantpornmovies.com, IP: 172.67.141.154
URL: wantpornmovies.com, IP: 172.67.141.154
URL: www.ktv.com.mx, IP: 172.67.141.154
URL: betvessel.co.uk, IP: 172.67.141.154
URL: www.betvessel.co.uk, IP: 172.67.141.154
URL: www.geodost.tv, IP: 104.21.71.153
URL: geodost.tv, IP: 104.21.71.153
URL: www.150tv.cn, IP: 104.21.71.153
URL: 150tv.cn, IP: 104.21.71.153
URL: stream.batperson.com, IP: 104.21.71.153
URL: altkattv.com.tr, IP: 104.21.71.153
URL: www.streamdeouf.vin, IP: 104.21.71.172
URL: allxvideos.net, IP: 104.21.71.172
URL: www.allxvideos.net, IP: 104.21.71.172
URL: jagomxwin88.tv, IP: 104.21.71.177
URL: t089.tv, IP: 104.21.71.177
URL: www.t089.tv, IP: 104.21.71.177
URL: axeliptv.aliui.me, IP: 104.21.71.177
URL: www.watchdaysales.com, IP: 104.21.71.177
URL: villagetv.org, IP: 104.21.71.177
URL: 83cwep.jstv2213.xyz, IP: 172.67.141.98
URL:

This results in more than 3700 domains for the search terms 'stream', 'video', 'movie', 'watch', 'tv'.

## DNS and Firewall Evasion (Time spent: xx h)



### DNS Mechanisms and Evasion Techniques

1. Why is DNS often used to bypass firewalls, and why is this a popular attack vector?

DNS is can be used to bypass firewalls because it operates over port 53, which is often allowed through firewalls for essential network functionality. This makes DNS traffic a convenient channel for attackers to tunnel malicious data, evade traditional firewall rules. The difficulty in inspecting DNS traffic thoroughly without specialized tools also contributes to its popularity as an attack vector. Robust DNS monitoring and filtering solutions are necessary to mitigate these risks.

2. Explain the process of how a DNS tunnel works from the client request through to the response.

A DNS tunnel works by encoding data into DNS queries sent by a client to a DNS server. The server extracts this data from the queries and sends back responses that contain the encoded information or acknowledgments. The client then decodes the data from the server's responses, enabling covert communication through DNS channels, therby bypassing security measures.

3. Describe a method to further cloak traffic via DNS tunneling. Provide a detailed description and analyze the overhead involved with concrete numbers and percentages.

    1. Payload Encryption: Encrypt data using AES-256 before embedding it into DNS queries to prevent interception. An attacker should be able to exchange keys in advance. This would add computational effort.
    2. Randomize query structure, timing, and data chunk order to evade pattern detection. Introduce random delays of 50-200 milliseconds between queries. This increases latency and network utilization with random delays.


### DoH, DoT Implementation and Analysis

1. Implement a subset of DNS over HTTPS (DoH) and DNS over TLS (DoT) to query an A record. Demonstrate the implementation by querying a public DNS server.

1.1 DNS over HTTPS Implementation:

In [12]:
import requests

def query_doh(domain):
    doh_url = "https://cloudflare-dns.com/dns-query"
    headers = {"Accept": "application/dns-json"}
    params = {"name": domain, "type": "A"}

    try:
        response = requests.get(doh_url, headers=headers, params=params)
        response.raise_for_status()
        answer = response.json()
        if "Answer" in answer:
            return answer["Answer"]
        else:
            print("No 'Answer' field in the response.")
            return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

domain = "example.com"
result = query_doh(domain)
if result:
    for record in result:
        print(f"{record['name']} has A record {record['data']}")
else:
    print("No A record found.")


example.com has A record 93.184.215.14


1.2 DNS over TLS Implementation:

Cant get it to run:
https://github.com/melvilgit/DNS-OVER-TLS/blob/master/test.py
https://community.cloudflare.com/t/python-ssl-for-connecting-with-cloudflare-1-1-1-1-853/33561

In [13]:
# Todo does not work :c BufferError 

import socket
import ssl
from dnslib import DNSRecord, A

SERVER_HOST = '1.1.1.1'
SERVER_PORT = 853

domain = 'example.com'
query = DNSRecord.question(domain, 'A')
query_bytes = query.pack()
message_length = len(query_bytes).to_bytes(2, byteorder='big')
query_with_length = message_length + query_bytes
context = ssl.create_default_context()

with socket.create_connection((SERVER_HOST, SERVER_PORT)) as sock:
    with context.wrap_socket(sock, server_hostname=SERVER_HOST) as ssock:
        ssock.send(query_with_length)
        response_bytes = b''
        while True:
            chunk = ssock.recv(4096)
            if not chunk:
                break
            response_bytes += chunk

'''A
response = DNSRecord.parse(response_bytes)
answers = response.rr
if answers:
    for rr in answers:
        if rr.rtype == A:
            print(f"{domain} -> {rr.rdata}")
else:
    print(f"No answers found for {domain}")
'''

'A\nresponse = DNSRecord.parse(response_bytes)\nanswers = response.rr\nif answers:\n    for rr in answers:\n        if rr.rtype == A:\n            print(f"{domain} -> {rr.rdata}")\nelse:\n    print(f"No answers found for {domain}")\n'

2. What are the drawbacks of DoH and DoT, and how could these drawbacks be addressed?

DoH and DoT could face firewall blocking. Additionally, encrypting DNS queries and responses can lead to higher latency. Moreover, adopting DoH might route traffic through a few major providers, potentially restricting DNS options.

To address these concerns, establishing standards and integrating DoH and DoT into current networks could provide solutions. Furthermore, encouraging diverse companies to offer DoH and DoT services would help prevent DNS centralization.